In [0]:
spark

In [0]:
strsource="wasbs://"+"staging"+"@"+"g7casestudy"+".blob.core.windows.net/"
strmountpoint="/mnt/staging"
strkey="fs.azure.account.key."+"g7casestudy"+".blob.core.windows.net"
strvalue="WBVvueMigJ4MFr/uL6gAipl9c9IZl/GBz1lH+GAtREBp2F0eOoy3qPNuw79mKZX075IrnTVTSSpR+AStzgUXfg=="
result=dbutils.fs.mount(
    source=strsource,
    mount_point=strmountpoint,
    extra_configs={strkey:strvalue}
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-2940978257572029>, line 5
      3 strkey="fs.azure.account.key."+"g7casestudy"+".blob.core.windows.net"
      4 strvalue="WBVvueMigJ4MFr/uL6gAipl9c9IZl/GBz1lH+GAtREBp2F0eOoy3qPNuw79mKZX075IrnTVTSSpR+AStzgUXfg=="
----> 5 result=dbutils.fs.mount(
      6     source=strsource,
      7     mount_point=strmountpoint,
      8     extra_configs={strkey:strvalue}
      9 )

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o5356.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/staging; nested exception is: 
	java.lang.IllegalArgu

In [0]:
dbutils.fs.ls('/mnt/staging')

[FileInfo(path='dbfs:/mnt/staging/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1696565150000),
 FileInfo(path='dbfs:/mnt/staging/_committed_6698972463593430581', name='_committed_6698972463593430581', size=864, modificationTime=1696565149000),
 FileInfo(path='dbfs:/mnt/staging/_committed_9096553128039432836', name='_committed_9096553128039432836', size=693, modificationTime=1696490548000),
 FileInfo(path='dbfs:/mnt/staging/_started_6698972463593430581', name='_started_6698972463593430581', size=0, modificationTime=1696565146000),
 FileInfo(path='dbfs:/mnt/staging/alt_fuel_biodiesel.csv', name='alt_fuel_biodiesel.csv', size=166790, modificationTime=1696566088000),
 FileInfo(path='dbfs:/mnt/staging/altfuelstations.csv', name='altfuelstations.csv', size=25881118, modificationTime=1696565550000),
 FileInfo(path='dbfs:/mnt/staging/part-00000-tid-6698972463593430581-94b6f5b6-4abd-4ce5-a60a-c7febd88903e-90-1-c000.csv', name='part-00000-tid-6698972463593430581-94b6f5b6-4abd-4ce5-a60a-c

In [0]:
df=spark.read.csv("/mnt/staging/altfuelstations.csv",inferSchema="true",header="True")

In [0]:
column_names = df.columns
print(column_names)


['Fuel Type Code', 'Station Name', 'Street Address', 'Intersection Directions', 'City', 'State', 'ZIP', 'Plus4', 'Station Phone', 'Status Code', 'Expected Date', 'Groups With Access Code', 'Access Days Time', 'Cards Accepted', 'BD Blends', 'NG Fill Type Code', 'NG PSI', 'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count', 'EV Other Info', 'EV Network', 'EV Network Web', 'Geocode Status', 'Latitude', 'Longitude', 'Date Last Confirmed', 'ID', 'Updated At', 'Owner Type Code', 'Federal Agency ID', 'Federal Agency Name', 'Open Date', 'Hydrogen Status Link', 'NG Vehicle Class', 'LPG Primary', 'E85 Blender Pump', 'EV Connector Types', 'Country', 'Intersection Directions (French)', 'Access Days Time (French)', 'BD Blends (French)', 'Groups With Access Code (French)', 'Hydrogen Is Retail', 'Access Code', 'Access Detail Code', 'Federal Agency Code', 'Facility Type', 'CNG Dispenser Num', 'CNG On-Site Renewable Source', 'CNG Total Compression Capacity', 'CNG Storage Capacity', 'LNG On-S

In [0]:
df.dropDuplicates()

DataFrame[Fuel Type Code: string, Station Name: string, Street Address: string, Intersection Directions: string, City: string, State: string, ZIP: string, Plus4: string, Station Phone: string, Status Code: string, Expected Date: string, Groups With Access Code: string, Access Days Time: string, Cards Accepted: string, BD Blends: string, NG Fill Type Code: string, NG PSI: string, EV Level1 EVSE Num: int, EV Level2 EVSE Num: string, EV DC Fast Count: string, EV Other Info: string, EV Network: string, EV Network Web: string, Geocode Status: string, Latitude: string, Longitude: string, Date Last Confirmed: string, ID: string, Updated At: string, Owner Type Code: string, Federal Agency ID: string, Federal Agency Name: string, Open Date: date, Hydrogen Status Link: string, NG Vehicle Class: string, LPG Primary: string, E85 Blender Pump: string, EV Connector Types: string, Country: string, Intersection Directions (French): string, Access Days Time (French): string, BD Blends (French): string,

In [0]:
##standardization
from pyspark.sql.functions import upper

df = df.withColumn('City', upper(df['City']))

In [0]:
from pyspark.sql.functions import col

df =df.withColumn('ZIP', col('ZIP').cast('integer'))


In [0]:
selected_df = df.select("Fuel Type Code", "Station Name", "Street Address", "City", "State", "ZIP", "Plus4", "Country")

# Show the selected DataFrame
selected_df.show()

+--------------+--------------------+--------------------+-------------+-----+-----+-----+-------+
|Fuel Type Code|        Station Name|      Street Address|         City|State|  ZIP|Plus4|Country|
+--------------+--------------------+--------------------+-------------+-----+-----+-----+-------+
|           CNG|Spire - Montgomer...|    2951 Chestnut St|   Montgomery|   AL|36107| NULL|     US|
|           CNG|Metropolitan Atla...| 2424 Piedmont Rd NE|      Atlanta|   GA|30324| NULL|     US|
|           CNG|United Parcel Ser...|270 Marvin Miller Dr|      Atlanta|   GA|30336| NULL|     US|
|           CNG|Arkansas Oklahoma...|   2100 S Waldron Rd|   Fort Smith|   AR|72903| NULL|     US|
|           CNG|Clean Energy - Lo...| 1000 Cottage St Ext|  East Boston|   MA| 2128| NULL|     US|
|           CNG|Clean Energy - Ev...|         16 Rover St|      Everett|   MA| 2149| NULL|     US|
|           CNG|Clean Energy - Gr...|     287 Maspeth Ave|     Brooklyn|   NY|11211| NULL|     US|
|         

In [0]:
selected_df.dropna()

DataFrame[Fuel Type Code: string, Station Name: string, Street Address: string, City: string, State: string, ZIP: string, Plus4: string, Country: string]

In [0]:
# Create a list of column names
from pyspark.sql.functions import col
column_names = selected_df.columns

# Use a for loop to iterate through the columns and count null values
for column_name in column_names:
    null_count = selected_df.filter(col(column_name).isNull()).count()
    print(f"Column '{column_name}': {null_count} null values")

Column 'Fuel Type Code': 0 null values
Column 'Station Name': 152 null values
Column 'Street Address': 124 null values
Column 'City': 348 null values
Column 'State': 388 null values
Column 'ZIP': 289 null values
Column 'Plus4': 77526 null values
Column 'Country': 468 null values


In [0]:
selected_df.createOrReplaceTempView("fuelstationstable")
df.createOrReplaceTempView("fuelstationstablemain")

In [0]:
##data analyisis with sql

In [0]:
#cities with the highest number of null values in the 'ZIP'

In [0]:
%sql
SELECT City, COUNT(*) AS null_zip_count
FROM fuelstationstable
WHERE ZIP IS NULL
GROUP BY City
ORDER BY null_zip_count DESC
LIMIT 10
OFFSET 1


City,null_zip_count
DE,4
WI,2
MA,2
AZ,1
DC,1
NC,1
NY,1
CO,1
FL,1
VA,1


In [0]:
##unique fuelt types


In [0]:
%sql
SELECT COUNT(DISTINCT `Fuel Type Code`) AS unique_fuel_types
FROM fuelstationstable;


unique_fuel_types
244


In [0]:
#State with the most alternate fuel stations.

In [0]:
%sql

select State,Count(ID) from fuelstationstablemain group by State order by Count(ID) desc limit 1



State,count(ID)
CA,18874


In [0]:
#trend of buying biodiesel state wise

In [0]:
%sql
select * from fuelstationstable where `Fuel Type Code` = 'BD'

Fuel Type Code,Station Name,Street Address,City,State,ZIP,Plus4,Country
BD,Dr. Dan's Alternative Fuels Werks,912 NW 50th St,Seattle,WA,98107,null,US
BD,Currituck County - North Carolina DOT,NC-1246,Maple,NC,27956,null,US
BD,Person County - North Carolina DOT,517 Leasburg Rd,Roxboro,NC,27574,null,US
BD,Durham County - North Carolina DOT,3910 Guess Rd,Durham,NC,27705,null,US
BD,Granville County - North Carolina DOT,5677 Cornwall Rd,Oxford,NC,27565,null,US
BD,Franklin County - North Carolina DOT,NC 39 Highway S,Bunn,NC,27508,null,US
BD,Vance County - North Carolina DOT,465 Gillburg Rd,Henderson,NC,27537,null,US
BD,Warren County - North Carolina DOT,State Route 1001,Warrenton,NC,27589,null,US
BD,Wake County Method Shop - North Carolina DOT,1301 Blue Ridge Rd,Raleigh,NC,27607,null,US
BD,Guilford County - North Carolina DOT,Sandy Camp Rd,High Point,NC,27265,null,US


In [0]:
out_path = "/mnt/curated/"

In [0]:
selected_df = selected_df.coalesce(1)

In [0]:
selected_df.write.csv(out_path, header=True, mode='overwrite')